In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import hiddenlayer as h
from visdom import Visdom
import datetime
import os

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary
from torchviz import make_dot

# 这里很普通的检查cuda可用性
x = torch.rand(2,2)
print(x)
print(torch.cuda.is_available())

tensor([[0.0252, 0.0055],
        [0.5658, 0.6703]])
True


In [2]:
ckpDir = './/ckp//c6'
if not os.path.exists(ckpDir):
    os.makedirs(ckpDir)

timeForSave = datetime.datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
print(timeForSave)
vizx = 0
viz = Visdom()
viz1 = Visdom()
viz2 = Visdom()
viz3 = Visdom()
viz4 = Visdom()

Setting up a new session...
Setting up a new session...


2021_12_05_16:31:34


Setting up a new session...
Setting up a new session...
Setting up a new session...


In [ ]:
# 以下两块是mat转numpy用的，训练时不需要运行，在此注掉了

datas = scio.loadmat('../data/Forpytorch20211205T095754.mat')
print(datas['segTR_X'].shape)
Xtrain = np.transpose(datas['segTR_X'],[3,2,0,1])
Xtest = np.transpose(datas['segTE_X'],[3,2,0,1])
Xtest_o = np.transpose(datas['segTE_Xo'],[3,2,0,1])
X_oo = np.transpose(datas['segTE_Xoo'],[3,2,0,1])

print(Xtrain.shape,'\n',Xtest.shape,'\n',\
    Xtest_o.shape,'\n',X_oo.shape)

# 标签
# print(datas1[0,0,:,:])
Ytrain = datas['segTR_Y']
Ytest = datas['segTE_Y']
Ytest_o = datas['segTE_Yo']
Y_oo = datas['segTE_Yoo']
print(Ytrain.shape,'\n',Ytest.shape,'\n',\
    Ytest_o.shape,'\n',Y_oo.shape)
print(Ytrain.shape,'\n',len(Ytrain),Ytrain[1,0])
# print(len(datalb))
# print(datas1[0,0,:,:])

# print(datas['Y_TrainP'].ndim)
# datalb = datas['Y_TrainP']
# print(len(datalb))
# print(datalb[1,0])
# emg1 = datalb[1,0]
# type(emg1)
# emg1.astype(np.float32)
# np.save('../data/trainY.npy',datalb)
# print(datalb[1,0])
dataset = {}
dataset['Ytrain'] = Ytrain
dataset['Xtrain'] = Xtrain
dataset['Xtest'] = Xtest
dataset['Xtest_o'] = Xtest_o
dataset['X_oo'] = X_oo
dataset['Ytest'] = Ytest
dataset['Ytest_o'] = Ytest_o
dataset['Y_oo'] = Y_oo
np.save('../data/OpenSetDataSet.npy',dataset)

In [3]:
# np.save('../data/trainX.npy',datas1)

In [4]:
# 1Ytrain-20210721T193346.mat

# datas = scio.loadmat('../data/Y_TrainP.mat')
# print(datas['Y_TrainP'].ndim)
# datalb = datas['Y_TrainP']
# print(len(datalb))
# print(datalb[1,0])
# emg1 = datalb[1,0]
# type(emg1)
# emg1.astype(np.float32)
# np.save('../data/trainY.npy',datalb)
# print(datalb[1,0])

In [5]:
# 这里检查了数据形状与维度，训练时不需要，注掉了

# tx = np.load('../data/trainX.npy')
# txp = tx[1,:,:,:]
# print(txp.shape)
# print(np.squeeze(txp).shape)

In [3]:
# 自定义数据集类

transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])

class EMGDataset(Dataset):
 
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.transforms = transform
 
    def __getitem__(self, index):
        emgData = self.data[index,:,:,:]
        emgData = np.squeeze(emgData)#似乎不应该压缩了
        emglabel = self.label[index]
        emglabel = emglabel.astype(np.int16)
        emgData = self.transforms(emgData)      
        
        return emgData,emglabel
 
    def __len__(self):
        return len(self.label)
 
 
# if __name__ == '__main__':
dataarray = np.load('../data/OpenSetDataSet.npy',allow_pickle=True)
CNNdataset = dataarray.item()
print(type(CNNdataset))
traindata = CNNdataset['Xtrain']
trainlabel = CNNdataset['Ytrain']
testdata = CNNdataset['Xtest']
testlabel = CNNdataset['Ytest']
# # print(trainlabel[:,0])

trainlabel = trainlabel[:,0]
testlabel = testlabel[:,0]
# print(type(trainlabel))
train_set = EMGDataset(traindata, trainlabel)
test_set = EMGDataset(testdata, testlabel)

# train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, pin_memory=True,
#                                             num_workers=3)

sample = next(iter(train_set))
print(sample)
sample = next(iter(test_set))
print(sample)

<class 'dict'>
(tensor([[[0.0171, 0.0073, 0.0024, 0.0024, 0.0024, 0.0024, 0.0757, 0.0317,
          0.0024, 0.0146],
         [0.0171, 0.0244, 0.0024, 0.0024, 0.0024, 0.0024, 0.0684, 0.0342,
          0.0024, 0.0146],
         [0.0146, 0.0269, 0.0024, 0.0024, 0.0024, 0.0024, 0.0610, 0.0342,
          0.0024, 0.0146],
         [0.0220, 0.0195, 0.0024, 0.0024, 0.0024, 0.0024, 0.0513, 0.0366,
          0.0024, 0.0098],
         [0.0269, 0.0171, 0.0024, 0.0024, 0.0024, 0.0024, 0.0488, 0.0415,
          0.0024, 0.0073],
         [0.0269, 0.0293, 0.0024, 0.0024, 0.0024, 0.0024, 0.0732, 0.0488,
          0.0024, 0.0073],
         [0.0244, 0.0317, 0.0024, 0.0024, 0.0024, 0.0024, 0.1001, 0.0635,
          0.0024, 0.0098],
         [0.0220, 0.0244, 0.0024, 0.0024, 0.0024, 0.0024, 0.1074, 0.0757,
          0.0024, 0.0195],
         [0.0171, 0.0146, 0.0024, 0.0024, 0.0024, 0.0024, 0.1172, 0.0854,
          0.0024, 0.0220],
         [0.0195, 0.0049, 0.0024, 0.0024, 0.0024, 0.0024, 0.1147, 0.0854,
 

In [4]:
# 自定义神经网络

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)

        self.fc1 = nn.Linear(in_features=32 * 4 * 34, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=6)
        self.dr1 = nn.Dropout2d(0.2)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        # t = self.dr1(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (4) hidden linear layer
        t = t.reshape(-1, 32 * 4 * 34)
        t = self.fc1(t)
        t = F.relu(t)
        t = self.dr1(t)

        # (5) output layer
        t = self.out(t)

        return t
        
net = Network()
# 打印网络，检查输入输出 shape是否正确
print(net)
summary(net,(1,40,10),batch_size = 1,device = "cpu")

# 可视化结构，hiddenlayer
# vis_graph = h.build_graph(net, torch.zeros([1,1,40,10]))
# vis_graph.theme = h.graph.THEMES["blue"].copy()
# vis_graph.save("./CNNtrynetframe")
# 可视化结构，torchviz
sampleInput = torch.randn(1,1,40,10).requires_grad_(True)
sampleOutput = net(sampleInput)
framevision = make_dot(sampleOutput, params=dict(list(net.named_parameters()) + [('x',sampleInput)]))
framevision.format = "png"
framevision.direcory = "./"
framevision.view()

Network(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4352, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=6, bias=True)
  (dr1): Dropout2d(p=0.2, inplace=False)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [1, 32, 38, 8]             320
            Conv2d-2             [1, 32, 35, 5]           9,248
            Linear-3                   [1, 128]         557,184
         Dropout2d-4                   [1, 128]               0
            Linear-5                     [1, 6]             774
Total params: 567,526
Trainable params: 567,526
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.

'Digraph.gv.png'

In [18]:
# import datetime

# timeForSave = datetime.datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
# print(timeForSave)
print(trainlabel.size)

2552


In [5]:
# # 训练过程可视化，hiddenlayer
# history = h.History()
# canvas = h.Canvas()

# 设置使用的训练设备
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
net = net.to(device)
# 加载数据，设置优化器
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1000,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100,shuffle=True)
optimizer = torch.optim.Adam(net.parameters(),
        lr=0.0002)
lr_schedule = torch.optim.lr_scheduler.StepLR(\
        optimizer, 400, gamma=0.5, last_epoch=-1)
total_test_acc = 0
total_test_correct = 0
totaltest = 0
vizx = 0
# 开始迭代
for epoch in range(50000):

    total_loss = 0
    total_correct = 0
    curr_total_correct = 0
    i = 0
    net.train()
    for batch in train_loader: # Get Batch
        # batch = next(iter(train_loader)) # Get Batch
        i+=1
        images, labels = batch
        # 数据和标签转为所需数据类型
        images = images.to(torch.float32)
        labels = labels.long()
        preds = net(images.to(device)) # Pass Batch
        trainloss = F.cross_entropy(preds.to(device), labels.to(device)) # Calculate Loss
        optimizer.zero_grad()
        trainloss.backward() # Calculate Gradients
        optimizer.step() # Update Weights
        total_loss += trainloss.item()
        # total_correct += get_num_correct(preds.to(device), labels.to(device))
        curr_total_correct = (get_num_correct(preds.to(device), labels.to(device)))
        total_correct += curr_total_correct
    total_train_acc = total_correct/(trainlabel.size)
    total_correct = 0

        # 可视化
    vizx+=1
    viz.line([float(trainloss)],[vizx],win='trainloss', update='append',opts=dict(title='trainloss'))
    
    viz2.line([float(optimizer.state_dict()['param_groups'][0]['lr'])],[vizx],\
        win='lr', update='append',opts=dict(title='lr'))
    viz3.line([float(total_train_acc)],[vizx],\
        win='train_acc', update='append',opts=dict(title='train_acc'))
    net.eval()
    total = 0
    for testemgdatas, testemglabels in test_loader: # Get Batch
        testemgdatas = testemgdatas.to(torch.float32)
        testemglabels = testemglabels.long()
        predstest = net(testemgdatas.to(device))
        testloss = F.cross_entropy(predstest.to(device), testemglabels.to(device)) # Calculate Loss
        curr_test_correct = (get_num_correct(predstest.to(device), testemglabels.to(device)))
        total += testemglabels.size(0)
        total_test_correct += curr_test_correct
        # totaltest += testemglabels.size(0)
    # total_test_acc = total_test_correct/(trainlabel.size)
    total_test_acc = total_test_correct/total
    
    viz1.line([float(testloss)],[vizx],win='testloss', update='append',opts=dict(title='testloss'))
    viz4.line([float(total_test_acc)],[vizx],\
        win='test_acc', update='append',opts=dict(title='test_acc'))
    total_test_correct = 0

    print(
        "epoch", epoch, 
        "total_train_acc:", total_train_acc, 
        "loss:", total_loss
    )
   # 更新学习率
    lr_schedule.step()
   # 定期保存
    if epoch%200 == 0:
        timeForSave = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
        checkpointPath = ckpDir+'c6_ep_'+str(epoch)+'_'+timeForSave+'.pth'
        state = {'model': net.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch}
        torch.save(state, checkpointPath)

torch.save(net.state_dict(),'./cnnet.pth')


epoch 0 total_train_acc: 0.19905956112852666 loss: 5.344899296760559
epoch 1 total_train_acc: 0.21826018808777428 loss: 5.165574669837952
epoch 2 total_train_acc: 0.22884012539184953 loss: 5.002880334854126
epoch 3 total_train_acc: 0.3221003134796238 loss: 4.821783185005188
epoch 4 total_train_acc: 0.38636363636363635 loss: 4.662320971488953
epoch 5 total_train_acc: 0.4306426332288401 loss: 4.489436149597168
epoch 6 total_train_acc: 0.4408307210031348 loss: 4.336328983306885
epoch 7 total_train_acc: 0.4431818181818182 loss: 4.201362729072571
epoch 8 total_train_acc: 0.46394984326018807 loss: 4.056785941123962
epoch 9 total_train_acc: 0.460423197492163 loss: 3.96440052986145
epoch 10 total_train_acc: 0.4862852664576803 loss: 3.872239828109741
epoch 11 total_train_acc: 0.5066614420062696 loss: 3.7754141092300415
epoch 12 total_train_acc: 0.4992163009404389 loss: 3.701851010322571
epoch 13 total_train_acc: 0.5223354231974922 loss: 3.6414469480514526
epoch 14 total_train_acc: 0.52821316614

KeyboardInterrupt: 

In [ ]:
# def openset_fuxin(dataloader, netC):
#     openset_scores = []
#     for i, (images, labels) in enumerate(dataloader):
#         images = Variable(images, volatile=True)
#         logits = netC(images)
#         augmented_logits = F.pad(logits, pad=(0,1))
#         # The implicit K+1th class (the open set class) is computed
#         #  by assuming an extra linear output with constant value 0
#         preds = F.softmax(augmented_logits)
#         #preds = augmented_logits
#         prob_unknown = preds[:, -1]
#         prob_known = preds[:, :-1].max(dim=1)[0]
#         prob_open = prob_unknown - prob_known

#         openset_scores.extend(prob_open.data.cpu().numpy())
#     return np.array(openset_scores)

In [10]:
# 重新加载模型
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)

        self.fc1 = nn.Linear(in_features=32 * 4 * 34, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=6)
        self.dr1 = nn.Dropout2d(0.2)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        # t = self.dr1(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (4) hidden linear layer
        t = t.reshape(-1, 32 * 4 * 34)
        t = self.fc1(t)
        t = F.relu(t)
        t = self.dr1(t)

        # (5) output layer
        t = self.out(t)

        return t
        
net_eval = Network()

checkpoint_eval = torch.load('./ckp/c6c6_ep_1600_2021_12_05_16_38_46.pth')
net_eval.load_state_dict(checkpoint_eval['model'])

<All keys matched successfully>

In [11]:
# 自定义数据集类

transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])

class EMGDataset(Dataset):
 
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.transforms = transform
 
    def __getitem__(self, index):
        emgData = self.data[index,:,:,:]
        emgData = np.squeeze(emgData)#似乎不应该压缩了
        emglabel = self.label[index]
        emglabel = emglabel.astype(np.int16)
        emgData = self.transforms(emgData)      
        
        return emgData,emglabel
 
    def __len__(self):
        return len(self.label)
 
 
# if __name__ == '__main__':
dataarray = np.load('../data/OpenSetDataSet.npy',allow_pickle=True)
CNNdataset = dataarray.item()
print(type(CNNdataset))
opensetdata = CNNdataset['X_oo']
opensetlabel = CNNdataset['Y_oo']
# trainlabel = CNNdataset['Ytrain']
# testdata = CNNdataset['Xtest']
# testlabel = CNNdataset['Ytest']
# # print(trainlabel[:,0])

opensetlabel = opensetlabel[:,0]
# testlabel = testlabel[:,0]
# print(type(trainlabel))
open_set = EMGDataset(opensetdata, opensetlabel)

# train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, pin_memory=True,
#                                             num_workers=3)
openset_loader = torch.utils.data.DataLoader(open_set, batch_size=1,shuffle=True)
sample = next(iter(openset_loader))
print(sample)

<class 'dict'>
[tensor([[[[0.0024, 0.0024, 0.0098, 0.0024, 0.0024, 0.0024, 0.0024, 0.0537,
           0.0024, 0.0024],
          [0.0024, 0.0024, 0.0146, 0.0024, 0.0024, 0.0024, 0.0024, 0.0488,
           0.0024, 0.0024],
          [0.0024, 0.0024, 0.0122, 0.0024, 0.0024, 0.0024, 0.0024, 0.0415,
           0.0024, 0.0024],
          [0.0024, 0.0024, 0.0073, 0.0024, 0.0024, 0.0024, 0.0024, 0.0439,
           0.0024, 0.0024],
          [0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0439,
           0.0024, 0.0024],
          [0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0513,
           0.0024, 0.0024],
          [0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0562,
           0.0024, 0.0024],
          [0.0073, 0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0024, 0.0586,
           0.0024, 0.0024],
          [0.0122, 0.0024, 0.0073, 0.0024, 0.0024, 0.0024, 0.0024, 0.0610,
           0.0024, 0.0024],
          [0.0098, 0.0024, 0.0098, 0.0024, 0.0024, 0.0024

In [19]:
openset_loader = torch.utils.data.DataLoader(open_set, batch_size=1,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1,shuffle=True)
openset_scores = []
openset_scores2 = []
# 可视化
viz6 = Visdom()
viz7 = Visdom()
vizx = 0
for opensetdata, opensetlabel in openset_loader: # Get Batch
        opensetdata = opensetdata.to(torch.float32)
        opensetlabel = opensetlabel.long()
        logits = net_eval(opensetdata)
        augmented_logits = F.pad(logits, pad=(0,1))
        preds = F.softmax(augmented_logits)
        prob_unknown = preds[:, -1]
        prob_known = preds[:, :-1].max(dim=1)[0]
        prob_open = prob_unknown - prob_known
        openset_scores.extend(prob_open.data.cpu().numpy())
        prob_openforvis = prob_open.float()
        viz6.line([float(prob_openforvis)],[vizx],\
                win='openscore', update='append',opts=dict(title='openscore for unknown'))
        vizx += 1

vizx = 0
for testemgdatas, testemglabels in test_loader: # Get Batch
        testemgdatas = testemgdatas.to(torch.float32)
        testemglabels = testemglabels.long()
        logits2 = net_eval(testemgdatas)
        augmented_logits2 = F.pad(logits2, pad=(0,1))
        preds2 = F.softmax(augmented_logits2)
        prob_unknown2 = preds2[:, -1]
        prob_known2 = preds2[:, :-1].max(dim=1)[0]
        prob_open2 = prob_unknown2 - prob_known2
        openset_scores2.extend(prob_open2.data.cpu().numpy())
        prob_openforvis2 = prob_open2.float()
        viz7.line([float(prob_openforvis2)],[vizx],\
                win='openscore2', update='append',opts=dict(title='openscore for known'))
        vizx += 1
# for i, (images, labels) in enumerate(dataloader):
#     images = Variable(images, volatile=True)
#     logits = netC(images)
#     augmented_logits = F.pad(logits, pad=(0,1))
#     # The implicit K+1th class (the open set class) is computed
#     #  by assuming an extra linear output with constant value 0
#     preds = F.softmax(augmented_logits)
#     #preds = augmented_logits
#     prob_unknown = preds[:, -1]
#     prob_known = preds[:, :-1].max(dim=1)[0]
#     prob_open = prob_unknown - prob_known
#     openset_scores.extend(prob_open.data.cpu().numpy())

Setting up a new session...
Setting up a new session...
C:\Users\cwdbo\AppData\Local\Temp/ipykernel_7396/609198364.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = F.softmax(augmented_logits)
C:\Users\cwdbo\AppData\Local\Temp/ipykernel_7396/609198364.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds2 = F.softmax(augmented_logits2)
